# Notebook 3: Backtesting du Modèle Morningstar

## Objectif
Charger le modèle entraîné (produit par le Notebook 02), exécuter un backtest sur un historique (dataset du Notebook 01), et analyser les performances avec des métriques claires et des graphiques d'évolution et d'évaluation.

## 1. Configuration Globale du Notebook

Modifiez les variables dans la cellule suivante pour configurer le notebook.

In [38]:
# --- Cellule 1 : Initialisation des modules et du projet ---
import json, glob, sys, os, logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Booléen pour activer / désactiver l’exécution du backtest
RUN_BACKTEST_SCRIPT = True

# Configuration des logs & graphiques
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (18, 7)
plt.rcParams["figure.dpi"]    = 100

# Ajouter le dossier 'ultimate' au PYTHONPATH pour importer run_backtest.py
PROJECT_ROOT      = os.path.abspath(os.getcwd())
PROJECT_CODE_ROOT = os.path.join(PROJECT_ROOT, "ultimate")
if PROJECT_CODE_ROOT not in sys.path:
    sys.path.append(PROJECT_CODE_ROOT)
    logger.info(f"Ajout de '{PROJECT_CODE_ROOT}' au PYTHONPATH.")

# Placeholders
run_backtest_main    = None
plot_drawdown_periods = None

# 1) Import du runner principal
try:
    from run_backtest import main as run_backtest_main
    logger.info("✅ run_backtest.main importé avec succès.")
except ImportError as e:
    logger.error(f"Impossible d’importer run_backtest.main : {e}", exc_info=True)

# 2) Import de la fonction de drawdown (fallback)
try:
    from backtesting.visualization import plot_drawdown_periods
    logger.info("✅ plot_drawdown_periods importé.")
except ImportError:
    try:
        from backtesting.visualization import plot_drawdown as plot_drawdown_periods
        logger.info("✅ plot_drawdown importé comme plot_drawdown_periods.")
    except ImportError:
        logger.warning("⚠️ plot_drawdown_periods introuvable : drawdown désactivé.")
        plot_drawdown_periods = None


2025-05-12 10:39:07,174 - __main__ - INFO - ✅ run_backtest.main importé avec succès.
2025-05-12 10:39:07,175 - __main__ - WARNING - ⚠️ plot_drawdown_periods introuvable : drawdown désactivé.


## 2. Vérification des Fichiers Prérequis

S'assure que le dataset enrichi (Notebook 01) et le modèle entraîné (Notebook 02) existent.

In [39]:
# --- Cellule 2 : Définition des chemins et vérifications ---
DATA_PATH    = os.path.join(PROJECT_CODE_ROOT, "data", "processed", "enriched_dataset.parquet")
MODEL_DIR    = os.path.join(PROJECT_CODE_ROOT, "outputs", "enhanced")
MODEL_FILE   = "best_model.keras"  # Nom par défaut
MODEL_PATH   = os.path.join(MODEL_DIR, MODEL_FILE)
RESULTS_DIR  = os.path.join(PROJECT_CODE_ROOT, "results", "backtest_notebook_output")
PAIR         = "BTC/USDT"           # Remplacez si besoin

# Création du répertoire de sortie
os.makedirs(RESULTS_DIR, exist_ok=True)

logger.info(f"DATA_PATH    = {DATA_PATH}")
logger.info(f"MODEL_PATH   = {MODEL_PATH}")
logger.info(f"RESULTS_DIR  = {RESULTS_DIR}")
logger.info(f"PAIR         = {PAIR}")

# Vérification d’existence
data_exists  = os.path.exists(DATA_PATH)
model_exists = os.path.exists(MODEL_PATH)

if not data_exists:
    logger.error(f"❌ Données manquantes : {DATA_PATH}")
    RUN_BACKTEST_SCRIPT = False

if not model_exists:
    logger.warning(f"⚠️ Modèle manquant : {MODEL_PATH} (backtest peut échouer)")

if not data_exists:
    logger.critical("🚨 Impossible de continuer sans données.")


2025-05-12 10:39:07,186 - __main__ - INFO - DATA_PATH    = /home/morningstar/Desktop/crypto_robot/Morningstar/ultimate/data/processed/enriched_dataset.parquet
2025-05-12 10:39:07,186 - __main__ - INFO - MODEL_PATH   = /home/morningstar/Desktop/crypto_robot/Morningstar/ultimate/outputs/enhanced/best_model.keras
2025-05-12 10:39:07,187 - __main__ - INFO - RESULTS_DIR  = /home/morningstar/Desktop/crypto_robot/Morningstar/ultimate/results/backtest_notebook_output
2025-05-12 10:39:07,187 - __main__ - INFO - PAIR         = BTC/USDT


## 3. Lancement du backtest

Appelle la fonction `main` de `ultimate/run_backtest.py` si `RUN_BACKTEST_SCRIPT` est `True` et que les données existent.

In [ ]:
# --- Cellule 3 : Lancement du backtest via run_backtest.py (flags ajustés) ---
if RUN_BACKTEST_SCRIPT and data_exists:
    logger.info(f"\n▶️ Lancement du backtest pour {PAIR}…")
    
    backtest_args = [
        "--data-dir",      os.path.join(PROJECT_CODE_ROOT, "data", "processed"),
        "--pair",          PAIR,
        "--model",         MODEL_PATH,
        "--results-dir",   RESULTS_DIR,
        "--initial-capital", str(20.0),
        "--commission",      str(0.002),
        "--slippage",        str(0.0005),
        "--threshold",       str(0.6),
    ]

    original_argv = sys.argv.copy()
    script_path   = os.path.join(PROJECT_CODE_ROOT, "run_backtest.py")
    sys.argv      = [script_path] + backtest_args

    logger.info(f"Appel : python {script_path} {' '.join(backtest_args)}")
    try:
        if run_backtest_main:
            exit_code = run_backtest_main()
            logger.info(f"✅ Backtest terminé (exit code {exit_code}).")
        else:
            logger.error("❌ run_backtest_main non importé : backtest sauté.")
    except SystemExit:
        logger.warning("⚠️ Backtest interrompu (argparse).")
    except Exception as e:
        logger.error(f"❌ Erreur durant le backtest : {e}", exc_info=True)
    finally:
        sys.argv = original_argv
else:
    logger.info("ℹ️ Backtest non lancé.")


2025-05-12 10:39:07,201 - __main__ - INFO - 
▶️ Lancement du backtest pour BTC/USDT…
2025-05-12 10:39:07,203 - __main__ - INFO - Appel : python /home/morningstar/Desktop/crypto_robot/Morningstar/ultimate/run_backtest.py --data-path /home/morningstar/Desktop/crypto_robot/Morningstar/ultimate/data/processed/enriched_dataset.parquet --pair BTC/USDT --model /home/morningstar/Desktop/crypto_robot/Morningstar/ultimate/outputs/enhanced/best_model.keras --results-dir /home/morningstar/Desktop/crypto_robot/Morningstar/ultimate/results/backtest_notebook_output --initial-capital 20.0 --commission 0.002 --slippage 0.0005 --threshold 0.6 --plot --loglevel INFO
usage: run_backtest.py [-h] --pair PAIR [--model MODEL] [--data-dir DATA_DIR]
                       [--results-dir RESULTS_DIR]
                       [--initial-capital INITIAL_CAPITAL]
                       [--commission COMMISSION] [--slippage SLIPPAGE]
                       [--threshold THRESHOLD]
run_backtest.py: error: unrecognized a

## 4. Lecture des résultats du Backtest

Charge les métriques JSON et le DataFrame de la courbe de capitaux produits par `run_backtest.py`.

In [41]:
# --- Cellule 4 : Recherche et affichage des résultats backtest ---
pf              = PAIR.replace("/", "").lower()
metrics_pattern = os.path.join(RESULTS_DIR, f"{pf}_backtest_metrics_*.json")
equity_pattern  = os.path.join(RESULTS_DIR, f"{pf}_equity_curve_*.csv")

logger.info(f"Recherche métriques : {metrics_pattern}")
logger.info(f"Recherche equity curve : {equity_pattern}")

metrics_files = sorted(glob.glob(metrics_pattern), key=os.path.getmtime, reverse=True)
equity_files  = sorted(glob.glob(equity_pattern),  key=os.path.getmtime, reverse=True)

# --- MÉTRIQUES JSON ---
if metrics_files:
    with open(metrics_files[0], "r") as f:
        metrics = json.load(f)
    print("\n--- MÉTRIQUES DU BACKTEST ---")
    print(json.dumps(metrics, indent=2))
else:
    logger.warning("⚠️ Aucun fichier métriques trouvé.")

# --- Equity Curve DataFrame ---
df_equity = None
if equity_files:
    df_equity = pd.read_csv(equity_files[0], index_col=0, parse_dates=True)
    print("\n--- Aperçu Equity Curve ---")
    display(df_equity.head())
else:
    logger.warning("⚠️ Aucun fichier equity curve trouvé.")


2025-05-12 10:39:07,242 - __main__ - INFO - Recherche métriques : /home/morningstar/Desktop/crypto_robot/Morningstar/ultimate/results/backtest_notebook_output/btcusdt_backtest_metrics_*.json
2025-05-12 10:39:07,243 - __main__ - INFO - Recherche equity curve : /home/morningstar/Desktop/crypto_robot/Morningstar/ultimate/results/backtest_notebook_output/btcusdt_equity_curve_*.csv
2025-05-12 10:39:07,245 - __main__ - WARNING - ⚠️ Aucun fichier métriques trouvé.
2025-05-12 10:39:07,246 - __main__ - WARNING - ⚠️ Aucun fichier equity curve trouvé.


## 5. Visualisation des Résultats du Backtest

Graphiques d'évolution de la performance (courbe de capitaux) et d'évaluation des drawdowns.

In [42]:
# --- Cellule 5 : Plot Equity & Drawdown ---
if df_equity is not None and not df_equity.empty:
    title = f"{PAIR} — Backtest Equity"

    # 1) Courbe d’équité
    fig, ax = plt.subplots()
    df_equity["equity"].plot(ax=ax, title=title)
    ax.set_ylabel("Equity ($)")
    plt.show()

    # 2) Périodes de drawdown
    if plot_drawdown_periods:
        fig, ax = plt.subplots()
        plot_drawdown_periods(df_equity["equity"], ax=ax, top_n=5, title="Drawdown — " + PAIR)
        plt.show()
    else:
        logger.warning("⚠️ plot_drawdown_periods non disponible.")
else:
    logger.warning("⚠️ Pas de DataFrame equity à afficher.")


2025-05-12 10:39:07,258 - __main__ - WARNING - ⚠️ Pas de DataFrame equity à afficher.


## 6. Analyse des Métriques Clés

Affichage clair et structuré des métriques de performance du backtest.

In [43]:
# --- Cellule 6 : Chargement & stats des trades ---
trades_pattern = os.path.join(RESULTS_DIR, f"{pf}_trades_list_*.csv")
trade_files    = sorted(glob.glob(trades_pattern), key=os.path.getmtime, reverse=True)

if trade_files:
    df_trades = pd.read_csv(trade_files[0], parse_dates=["opendt", "closedt"])
    logger.info(f"✅ Trades chargés ({len(df_trades)} lignes) depuis : {trade_files[0]}")
else:
    logger.warning(f"⚠️ Aucun fichier trades trouvé ({trades_pattern}).")
    df_trades = pd.DataFrame()

# --- Période & wallet initial/final ---
if df_equity is not None and not df_equity.empty:
    start_date      = df_equity.index.min()
    end_date        = df_equity.index.max()
    initial_wallet  = df_equity["equity"].iloc[0]
    final_wallet    = df_equity["equity"].iloc[-1]
    performance_pct = (final_wallet / initial_wallet - 1) * 100

    # Buy & hold (dernier prix vs premier prix)
    df_prices = pd.read_parquet(DATA_PATH, columns=["timestamp", "close"]) \
                  .set_index("timestamp").sort_index()
    bh_return = (df_prices["close"].iloc[-1] / df_prices["close"].iloc[0] - 1) * 100

    print(f"Period: {start_date.date()} → {end_date.date()}")
    print(f"Initial wallet: {initial_wallet:.2f} $")
    print(f"Final wallet:   {final_wallet:.2f} $")
    print(f"Performance:    {performance_pct:.2f} %")
    print(f"Buy & Hold:     {bh_return:.2f} %")
    print(f"VS Buy&Hold:    {performance_pct - bh_return:.2f} %")

# --- Stats de trades détaillées ---
if not df_trades.empty:
    total_trades      = len(df_trades)
    days              = max((end_date - start_date).days, 1)
    trades_per_day    = total_trades / days
    df_trades["pnl_pct"] = (df_trades["closeprice"] / df_trades["openprice"] - 1) * 100

    winners = df_trades[df_trades["pnl_pct"] > 0]
    losers  = df_trades[df_trades["pnl_pct"] < 0]

    print("\n--- Trades Summary ---")
    print(f"Total trades:            {total_trades}")
    print(f"Trades per day (avg):     {trades_per_day:.2f}")
    print(f"Win rate:                {len(winners)/total_trades*100:.2f} %")
    print(f"Avg PnL (+):             {winners['pnl_pct'].mean():.2f} %")
    print(f"Avg PnL (−):             {losers['pnl_pct'].mean():.2f} %")
else:
    logger.warning("⚠️ Pas de trades à résumer.")


2025-05-12 10:39:07,280 - __main__ - WARNING - ⚠️ Aucun fichier trades trouvé (/home/morningstar/Desktop/crypto_robot/Morningstar/ultimate/results/backtest_notebook_output/btcusdt_trades_list_*.csv).
2025-05-12 10:39:07,284 - __main__ - WARNING - ⚠️ Pas de trades à résumer.


In [44]:
# --- Cellule 7 : Top 5 symbols par gain cumulé ---
if not df_trades.empty:
    agg = (
        df_trades
        .groupby("symbol")["pnl_pct"]
        .agg(Total="% sum", Avg="mean", Min="min", Max="max", Count="count")
        .rename(columns={"Total":"Total %", "Avg":"Avg %", "Min":"Min %", "Max":"Max %", "Count":"N trades"})
        .sort_values("Total %", ascending=False)
    )
    display(agg.head(5))
else:
    logger.warning("⚠️ Pas de trades pour afficher le top symbols.")


2025-05-12 10:39:07,299 - __main__ - WARNING - ⚠️ Pas de trades pour afficher le top symbols.


Cellule 7 – Rapport détaillé

In [45]:
# --- Cellule 8 : Top 5 symbols par gain cumulatif ---
if df_trades is not None and not df_trades.empty:
    agg = (
        df_trades
        .groupby('symbol')['pnl_pct']
        .agg(Total='% sum', Avg='% mean', Min='% min', Max='% max', N='count')
        .rename(columns={'Total':'Total %','Avg':'Avg %','Min':'Min %','Max':'Max %','N':'N trades'})
        .sort_values('Total %', ascending=False)
    )
    display(agg.head(5))
else:
    logger.warning("Pas de trades pour afficher le top symbols.")


2025-05-12 10:39:07,310 - __main__ - WARNING - Pas de trades pour afficher le top symbols.
